<a href="https://colab.research.google.com/github/mbribiescaucla-afk/imperial-valley-et/blob/main/(16_23)ETSeasonal_fromALF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. INSTALACIÓN Y AUTENTICACIÓN ---
!pip install -q earthengine-api geopandas

import geopandas as gpd
import pandas as pd
import ee
import json
import os
import time
from google.colab import drive

drive.mount('/content/drive')
ee.Authenticate()
ee.Initialize(project='ee-mbribiescaucla')

Mounted at /content/drive


In [2]:
# --- 2. CARGAR SHAPEFILE ---
shapefile_path = '/content/drive/MyDrive/Shapefiles/CSB_IV_16_23.shp'
gdf = gpd.read_file(shapefile_path)

for y in range(2016, 2024):
    gdf[f'CDL{y}'] = pd.to_numeric(gdf[f'CDL{y}'], errors='coerce')

# --- 3. IDENTIFICAR PATRONES DE CAMBIO DESDE ALFALFA ---
pattern_results = []

for n_initial36 in range(7, 0, -1):
    n_finalO = 8 - n_initial36
    initial36_years = [f'CDL{2016 + i}' for i in range(n_initial36)]
    finalO_years = [f'CDL{2016 + i}' for i in range(n_initial36, 8)]
    pattern_label = 'A' * n_initial36 + 'O' * n_finalO
    required_cols = initial36_years + finalO_years
    gdf_clean = gdf.dropna(subset=required_cols)

    def match_pattern(row):
        if any(row[y] != 36 for y in initial36_years): return False
        if any(row[y] == 36 for y in finalO_years): return False
        return True

    matched = gdf_clean[gdf_clean.apply(match_pattern, axis=1)].copy()
    matched['pattern'] = pattern_label
    pattern_results.append(matched)

# También agregamos el patrón AAAAAAAA (solo alfalfa)
all_years = [f'CDL{2016 + i}' for i in range(8)]
gdf_clean = gdf.dropna(subset=all_years)

def match_all_A(row):
    return all(row[y] == 36 for y in all_years)

matched_all_A = gdf_clean[gdf_clean.apply(match_all_A, axis=1)].copy()
matched_all_A['pattern'] = 'AAAAAAAA'
pattern_results.append(matched_all_A)

# Combinar todos los patrones encontrados
gdf_all = pd.concat(pattern_results)
print("Total de polígonos seleccionados:", len(gdf_all))

Total de polígonos seleccionados: 2116


In [3]:
# --- 4. EXPORTAR GEOJSON A EE ---
gdf_all = gdf_all.to_crs("EPSG:4326")
geojson_path = '/content/all_patterns_from_alfalfa.json'
if os.path.exists(geojson_path): os.remove(geojson_path)
gdf_all.to_file(geojson_path, driver='GeoJSON')

with open(geojson_path) as f:
    geojson_data = json.load(f)

features = [ee.Feature(ee.Geometry(f['geometry']), f['properties']) for f in geojson_data['features']]
fc = ee.FeatureCollection(features)
print("Polígonos cargados en EE:", fc.size().getInfo())

Polígonos cargados en EE: 2116


In [4]:
# --- 5. CALCULAR ET DIARIA PROMEDIO POR ESTACIÓN ---
results_list = []

modelos = ['ensemble', 'disalexi', 'eemetric']
colecciones = {
    'ensemble': ('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'et_ensemble_sam'),
    'disalexi': ('OpenET/DISALEXI/CONUS/GRIDMET/MONTHLY/v2_0', 'et'),
    'eemetric': ('OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0', 'et')
}

seasons = {
    'Winter': {'months': [12, 1, 2], 'days': {12: 31, 1: 31, 2: 28}},
    'Spring': {'months': [3, 4, 5], 'days': {3: 31, 4: 30, 5: 31}},
    'Summer': {'months': [6, 7, 8], 'days': {6: 30, 7: 31, 8: 31}},
    'Fall':   {'months': [9, 10, 11], 'days': {9: 30, 10: 31, 11: 30}}
}

for year in range(2016, 2024):
    print(f"🔄 Año {year}...")

    for season, info in seasons.items():
        months = info['months']
        total_days = sum(info['days'][m] for m in months)

        if season == 'Winter':
            start = ee.Date.fromYMD(year - 1, 12, 1)
            end = ee.Date.fromYMD(year, 2, 28)
        else:
            start = ee.Date.fromYMD(year, months[0], 1)
            end = ee.Date.fromYMD(year, months[-1], info['days'][months[-1]])

        for model in modelos:
            print(f"  ➤ Modelo: {model.upper()} - Estación: {season}")
            collection_id, band_name = colecciones[model]
            col = ee.ImageCollection(collection_id).filterDate(start, end)
            image = col.select(band_name).sum()

            reduced = image.reduceRegions(
                collection=fc,
                reducer=ee.Reducer.mean(),
                scale=30
            ).map(lambda f: f.set('year', year)
                          .set('season', season)
                          .set('model', model.upper())
                          .set('et_mm_day', ee.Number(f.get('mean')).divide(total_days)))

            results_list.append(reduced)

all_results = ee.FeatureCollection(results_list).flatten()

🔄 Año 2016...
  ➤ Modelo: ENSEMBLE - Estación: Winter
  ➤ Modelo: DISALEXI - Estación: Winter
  ➤ Modelo: EEMETRIC - Estación: Winter
  ➤ Modelo: ENSEMBLE - Estación: Spring
  ➤ Modelo: DISALEXI - Estación: Spring
  ➤ Modelo: EEMETRIC - Estación: Spring
  ➤ Modelo: ENSEMBLE - Estación: Summer
  ➤ Modelo: DISALEXI - Estación: Summer
  ➤ Modelo: EEMETRIC - Estación: Summer
  ➤ Modelo: ENSEMBLE - Estación: Fall
  ➤ Modelo: DISALEXI - Estación: Fall
  ➤ Modelo: EEMETRIC - Estación: Fall
🔄 Año 2017...
  ➤ Modelo: ENSEMBLE - Estación: Winter
  ➤ Modelo: DISALEXI - Estación: Winter
  ➤ Modelo: EEMETRIC - Estación: Winter
  ➤ Modelo: ENSEMBLE - Estación: Spring
  ➤ Modelo: DISALEXI - Estación: Spring
  ➤ Modelo: EEMETRIC - Estación: Spring
  ➤ Modelo: ENSEMBLE - Estación: Summer
  ➤ Modelo: DISALEXI - Estación: Summer
  ➤ Modelo: EEMETRIC - Estación: Summer
  ➤ Modelo: ENSEMBLE - Estación: Fall
  ➤ Modelo: DISALEXI - Estación: Fall
  ➤ Modelo: EEMETRIC - Estación: Fall
🔄 Año 2018...
  ➤ Modelo

In [5]:
# --- 6. EXPORTAR A DRIVE ---
export_desc = 'ET_daily_seasonal_from_alfalfa_Ensemble_DisAlexi_eeMETRIC_2016_2023'
task = ee.batch.Export.table.toDrive(
    collection=all_results,
    description=export_desc,
    folder='Colab_ET',
    fileFormat='CSV'
)
task.start()

while task.active():
    print("Esperando exportación...")
    time.sleep(10)

print("✅ Exportación completada:", task.status())

Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
✅ Exportación completada: {'state': 'COMPLETED', 'description': 'ET_daily_seasonal_from_alfalfa_Ensemble_DisAlexi_eeMETRIC_2016_2023', 'priority': 100, 'creation_timestamp_ms': 1755720053016, 'update_timestamp_ms': 1755720218137, 'start_timestamp_ms': 1755720062566, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1gwo7ys-IUcg4JYI4ytcC5_hgJRJdymR3'], 'attempt': 1, 'batch_eecu_usage_seconds': 2388.396240234375, 'id': 'TNNM4NWQ2Y53OECY54Y367PO', 'name': 'projects/ee-mbribiescaucla/operations/TNNM4NWQ2Y53OECY54Y367PO'}


In [6]:
# --- 7. CARGAR Y PIVOTEAR EN PYTHON ---
ruta_csv = '/content/drive/MyDrive/Colab_ET/ET_daily_seasonal_from_alfalfa_Ensemble_DisAlexi_eeMETRIC_2016_2023.csv'
df = pd.read_csv(ruta_csv)

# Filtrar solo modelo ENSEMBLE
df = df[df['model'] == 'ENSEMBLE'].copy()

# Año como texto con últimos 2 dígitos
df['year'] = df['year'].astype(str).str[-2:]

# Crear columna combinada
df['Season_Year'] = df['season'] + '_' + df['year']

# Pivotear DataFrame
pivot_df = df.pivot_table(
    index=['CSBID', 'AreaRec', 'pattern', 'CDL2016', 'CDL2017', 'CDL2018', 'CDL2019',
           'CDL2020', 'CDL2021', 'CDL2022', 'CDL2023'],
    columns='Season_Year',
    values='et_mm_day'
)

# Orden de columnas
orden_estaciones = ['Fall', 'Winter', 'Spring', 'Summer']
orden_columnas = sorted(pivot_df.columns, key=lambda x: (orden_estaciones.index(x.split('_')[0]), int(x.split('_')[1])))
pivot_df = pivot_df[orden_columnas]

# Renombrar columnas
pivot_df.columns = ['ET Ensemble ' + c for c in pivot_df.columns]

# Exportar a Excel
pivot_df.reset_index(inplace=True)
salida = '/content/drive/MyDrive/Colab_ET/ET_ENS_Pivoted_from_alfalfa_2016_2023.xlsx'
pivot_df.to_excel(salida, index=False)

print("✅ Archivo exportado:", salida)

✅ Archivo exportado: /content/drive/MyDrive/Colab_ET/ET_ENS_Pivoted_from_alfalfa_2016_2023.xlsx


In [7]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# --- 1. CARGAR ARCHIVO ---
ruta = '/content/drive/MyDrive/Colab_ET/ET_ENS_Pivoted_from_alfalfa_2016_2023.xlsx'
df = pd.read_excel(ruta)

# --- 2. FILTRAR POLÍGONOS CON CDL != 37 y 61 EN TODOS LOS AÑOS ---
cdl_cols = [f'CDL{y}' for y in range(2016, 2024)]
df = df[~df[cdl_cols].isin([37, 61]).any(axis=1)].copy()

# --- 3. IDENTIFICAR COLUMNAS POR ESTACIÓN ---
estaciones = ['Fall', 'Winter', 'Spring', 'Summer']
cols_por_estacion = {s: [c for c in df.columns if f'ET Ensemble {s}_' in c] for s in estaciones}

# --- 4. CALCULAR PROMEDIO ANUAL POR POLÍGONO ---
df['Annual'] = df[[col for cols in cols_por_estacion.values() for col in cols]].mean(axis=1)

# --- 5. CALCULAR PROMEDIO POR ESTACIÓN (8 AÑOS) ---
for est, cols in cols_por_estacion.items():
    df[est] = df[cols].mean(axis=1)

# --- 6. AGRUPAR POR PATRÓN Y CALCULAR ESTADÍSTICAS ---
estadisticas = []

for pattern, group in df.groupby('pattern'):
    resumen = {'Pattern': pattern}
    for est in ['Annual', 'Fall', 'Winter', 'Spring', 'Summer']:
        valores = group[est]
        resumen[f'{est}_Max'] = valores.max()
        resumen[f'{est}_Min'] = valores.min()
        resumen[f'{est}_Avg'] = valores.mean()
        resumen[f'{est}_Std'] = valores.std()
    estadisticas.append(resumen)

# --- 7. CONVERTIR A DATAFRAME Y FORMATEAR TABLA FINAL ---
tabla = pd.DataFrame(estadisticas)
tabla.set_index('Pattern', inplace=True)
tabla = tabla.sort_index()

# --- 8. FORMATO FINAL (COMO IMAGEN) ---
final = []

for pattern in tabla.index:
    for stat in ['Max', 'Min', 'Avg', 'Std']:
        fila = {'Pattern': pattern, 'Stat': {
            'Max': 'Maximum', 'Min': 'Minimum',
            'Avg': 'Average', 'Std': 'Std. Dev'
        }[stat]}
        for est in ['Annual', 'Fall', 'Winter', 'Spring', 'Summer']:
            fila[est] = tabla.loc[pattern, f'{est}_{stat}']
        final.append(fila)

tabla_final = pd.DataFrame(final)

# --- 9. EXPORTAR A EXCEL ---
salida = '/content/drive/MyDrive/Colab_ET/Resumen_ET_Ensemble_filtrado.xlsx'
tabla_final.to_excel(salida, index=False)
print("✅ Tabla exportada:", salida)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Tabla exportada: /content/drive/MyDrive/Colab_ET/Resumen_ET_Ensemble_filtrado.xlsx
